<a href="https://colab.research.google.com/github/WoojinJeonkr/DeepLearning/blob/main/TFRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TFRS
- 추천자 시스템 모델을 빌드하기 위한 라이브러리
- 추천자 시스템을 빌드하는 전체 워크플로(데이터 준비, 모델 공식화, 학습, 평가 및 배포)에 도움이 된다
- 목표: Keras에 기반하며 복잡한 모델을 빌드하는 유연성을 제공하면서 완만한 학습 곡선을 갖추는 것
- 코드 출처: https://www.tensorflow.org/recommenders/examples/quickstart?hl=ko

## 01. TFRS 설치

In [1]:
!pip install -q tensorflow-recommenders

     |████████████████████████████████| 88 kB 2.7 MB/s 
     |████████████████████████████████| 511.7 MB 5.9 kB/s 
     |████████████████████████████████| 1.6 MB 41.1 MB/s 
     |████████████████████████████████| 5.8 MB 30.0 MB/s 
     |████████████████████████████████| 438 kB 42.3 MB/s 


In [2]:
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 4.3 MB 5.1 MB/s 


## 02. 라이브러리 불러오기

In [3]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

## 03. 데이터 읽기

In [5]:
# 데이터 분할
ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteB5OXGG/movielens-train.tfrecord*...:   0…

Dataset movielens downloaded and prepared to ~/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [6]:
# 레이어를 포함하기 위해 사용자 ID와 영화 제목을 정수 인덱스로 변환하는 어휘 구축
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

## 04. 모델 정의

In [7]:
class MovieLensModel(tfrs.Model):
  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    self.user_model = user_model
    self.movie_model = movie_model

    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

In [8]:
# 검색 작업 정의
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

In [9]:
# 모델 생성
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# 3번 반복하여 훈련
model.fit(ratings.batch(4096), epochs=3)

index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/3
25/25 [==============================] - 33s 1s/step - factorized_top_k/top_1_categorical_accuracy: 6.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0048 - factorized_top_k/top_50_categorical_accuracy: 0.0443 - factorized_top_k/top_100_categorical_accuracy: 0.1033 - loss: 33081.6763 - regularization_loss: 0.0000e+00 - total_loss: 33081.6763
Epoch 2/3
25/25 [==============================] - 30s 1s/step - factorized_top_k/top_1_categorical_accuracy: 2.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0051 - factorized_top_k/top_10_categorical_accuracy: 0.0146 - factorized_top_k/top_50_categorical_accuracy: 0.1060 - factorized_top_k/top_100_categorical_accuracy: 0.2114 - loss: 31016.6855 - regularization_loss: 0.0000e+00 - total_loss: 31016.6855
Epoch 3/3
25/25 [==============================] - 30s 1s/step - factorized_top_k/top_1_categorical_accuracy: 5.7000e-04 - factorized_top_k/top_5_categorical_accu